In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, metrics, ensemble
from sklearn import model_selection
from xgboost import XGBClassifier
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def read_file(filename):
    raw_data = pd.read_csv(filename)
    data = raw_data.copy()
    return data

In [3]:
# read in train and test data -> X_train, y_train, X_test (pd.df)
train_path = "train_2008.csv"
test_path = "test_2008.csv"
train_data = read_file(train_path)
test_data = read_file(test_path)

y_train = train_data['target']
X_train = train_data.drop('target', axis=1)
X_test = test_data

In [4]:
# drop same_resp feature
def select_same_resp_feature(X_train):
    same_resp_feature = []
    for feature in X_train.columns:
        if len(np.unique(X_train[feature])) == 1:
            same_resp_feature.append(feature)
    return same_resp_feature

same_resp_feature = select_same_resp_feature(X_train)
X_train.drop(same_resp_feature, axis=1, inplace=True)
X_test.drop(same_resp_feature, axis=1, inplace=True)
print(len(same_resp_feature))
print(same_resp_feature)

15
['HRMONTH', 'HRYEAR4', 'HUTYPEA', 'HUTYPC', 'HRINTSTA', 'PEAFNOW', 'PRPERTYP', 'PULKDK4', 'PULKDK5', 'PULKDK6', 'PULKPS4', 'PULKPS5', 'PULKPS6', 'HXPHONEO', 'PXAGE']


In [5]:
# drop low resp features
response_rates = X_train[X_train >= 0].count() / len(X_train)
mostly_blank_feats = []
for itm in response_rates.items():
    if itm[1] < 0.01:
#         X_train[feature] = X_train[feature].apply(lambda x: np.NaN if (x == -1) else x)
#         X_test[feature] = X_test[feature].apply(lambda x: np.NaN if (x == -1) else x)
        mostly_blank_feats.append(itm[0])
print(len(mostly_blank_feats))

X_train.drop(mostly_blank_feats, axis=1, inplace=True)
X_test.drop(mostly_blank_feats, axis=1, inplace=True)

58


In [6]:
# encode
# s = X_train['HUFINAL']
# out = s.value_counts(normalize=True)
# print(out)
# print(type(out))
# out.max() > 0.9
# maxx = out.idxmax()
# X_train['HUFINAL']= X_train['HUFINAL'].apply(lambda x: 1 if (x == maxx) else 0)
# print(X_train['HUFINAL'].values[:10])

# recode
for i in X_train.columns:
    cols = X_train[i]
    print(type(cols))
    #print(cols)
    freq_out = cols.value_counts(normalize=True)
    if freq_out.max() > 0.95:
        max_idx = freq_out.idxmax()
        X_train[i] = X_train[i].apply(lambda x: 1 if (x == max_idx) else 0)
        X_test[i] = X_test[i].apply(lambda x: 1 if (x == max_idx) else 0)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

In [ ]:
print(X_train.values.shape)

In [9]:
print(X_train.values[:10, :10])

[[16000     1   201     1     1     1     1     1     1    16]
 [16001     1   201     1     2     1     1     1     1     6]
 [16002     1     1     1     1     1     1     1     1    15]
 [16003     2     1     1     1     1     1     1     1    13]
 [16004     2   201     1     1     1     1     1     1    -3]
 [16005     2   201     1     1     0     1     1     1    13]
 [16006     2   201     1     1     1     1     1     1    -3]
 [16007     1   201     1     2     1     0     0     1     7]
 [16008     1   201     1     3     0     1     1     1     1]
 [16009     1   201     1     1     1     1     1     1    13]]


In [7]:
# negative -> -1
for feature in X_train.columns:
    X_train[feature] = X_train[feature].apply(lambda x: np.NaN if (x == -1) else x)
    X_test[feature] = X_test[feature].apply(lambda x: np.NaN if (x == -1) else x)

In [8]:
# encode some features
categorical_features = ['HUFINAL','HEHOUSUT','GEREG', 'HUBUS', 'PTDTRACE', 'PENATVTY', 'PUABSOT', 'PEIO1COW', 
                     'HUFINAL', 'GESTCEN', 'GESTFIPS', #'PEIO1ICD', 'PEIO2ICD', 
                     'PRCITSHP', 'PUDIS', 'PRABSREA', 'PRWKSTAT', 'HUPRSCNT', 
                     'PERRP', 'GTCBSAST', 'PRMJOCGR', 'HRHTYPE', ]
# Now dummy these features
train_dummy_df = pd.DataFrame()
test_dummy_df = pd.DataFrame()

new_cat = []
for feature in categorical_features:
    if feature in X_train.columns:
        new_cat.append(feature)
        train_dummy_vars = pd.get_dummies(X_train[feature], prefix=feature)
        train_dummy_df = pd.concat([train_dummy_df, train_dummy_vars], axis=1)

        test_dummy_vars = pd.get_dummies(X_test[feature], prefix=feature)
        test_dummy_df = pd.concat([test_dummy_df, test_dummy_vars], axis=1)
# Drop the original categorical variables
X_train.drop(new_cat, axis=1, inplace=True)
X_test.drop(new_cat, axis=1, inplace=True)
# Add dummy vars to the data
X_train = pd.concat([X_train, train_dummy_df], axis=1)
X_test = pd.concat([X_test, test_dummy_df], axis=1)

# train_test 
feats_to_add_to_train = [f for f in X_test.columns if f not in X_train.columns]
feats_to_add_to_test = [f for f in X_train.columns if f not in X_test.columns]
for feat in feats_to_add_to_train:
    X_train[feat] = np.nan
for feat in feats_to_add_to_test:
    X_test[feat] = np.nan

In [5]:
    
X = X_train.values
Y = y_train.values
X_t = X_test.values
print(X.shape)
print(Y.shape)
print(X_t.shape)

(64667, 367)
(64667,)
(16000, 367)


In [30]:
print(X[:10, :10])

[[1.6000e+04 1.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 1.6000e+01]
 [1.6001e+04 1.0000e+00 2.0100e+02 0.0000e+00 2.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 6.0000e+00]
 [1.6002e+04 1.0000e+00 1.0000e+00 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 1.5000e+01]
 [1.6003e+04 2.0000e+00 1.0000e+00 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00 1.3000e+01]
 [1.6004e+04 2.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00        nan]
 [1.6005e+04 2.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 6.0000e+00
  1.0000e+00        nan 1.0000e+00 1.3000e+01]
 [1.6006e+04 2.0000e+00 2.0100e+02 0.0000e+00 1.0000e+00 1.0000e+00
  1.0000e+00        nan 1.0000e+00        nan]
 [1.6007e+04 1.0000e+00 2.0100e+02 0.0000e+00 2.0000e+00 1.0000e+00
  2.0000e+00 2.0000e+00 1.0000e+00 7.0000e+00]
 [1.6008e+04 1.0000e+00 2.0100e+02 0.0000e+00 3.0000e+00 5.0000e+00
  1.0000e+00

In [10]:
test_size = 0.3
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=10)
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.7, min_child_weight=2, n_estimators=50, objective='binary:logistic') 
clf_xgb.fit(X_train, y_train)
y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
test_auc = metrics.roc_auc_score(y_test, y_test_pred)
print("test auc: ", test_auc)

train auc:  0.8724250633754376
test auc:  0.7920151734759794


In [11]:
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgb_test_po2008.csv")

In [16]:
test_size = 0.3
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=10)
clf_xgb = XGBClassifier(max_depth=8, gamma=0.3, subsample=0.8, min_child_weight=2, n_estimators=50, objective='binary:logistic', scale_pos_weight=0.33) 
clf_xgb.fit(X_train, y_train)
y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
test_auc = metrics.roc_auc_score(y_test, y_test_pred)
print("test auc: ", test_auc)

train auc:  0.8542491663166532
test auc:  0.7895405568612743


In [15]:
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgb_test_po2_2008.csv")

In [33]:
test_size = 0.3
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=10)
clf_xgb = XGBClassifier(max_depth=9, gamma=0.2, subsample=0.8, min_child_weight=2, n_estimators=250, objective='binary:logistic', eta=0.05) 
clf_xgb.fit(X_train, y_train)
y_train_pred = clf_xgb.predict_proba(X_train)[:, 1]
y_test_pred = clf_xgb.predict_proba(X_test)[:, 1]

train_auc = metrics.roc_auc_score(y_train, y_train_pred)
print("train auc: ", train_auc)
test_auc = metrics.roc_auc_score(y_test, y_test_pred)
print("test auc: ", test_auc)

train auc:  0.9841290027468171
test auc:  0.7895549181000657


In [6]:
#test_size = 0.3
#X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=10)
clf_xgb = XGBClassifier(max_depth=9, gamma=0.2, subsample=0.8, min_child_weight=2, n_estimators=250, objective='binary:logistic', eta=0.05) 
clf_xgb.fit(X, Y)
y_train_pred = clf_xgb.predict_proba(X)[:, 1]
train_auc = metrics.roc_auc_score(Y, y_train_pred)
print("train auc: ", train_auc)

train auc:  0.9726203389790439


In [8]:
y_test_pred_prob = clf_xgb.predict_proba(X_t)[:, 1]
pd.DataFrame(y_test_pred_prob).to_csv("xgb_test_po3_2008.csv")
#test_auc = metrics.roc_auc_score(Y, y_train_pred)
#print("test auc: ", test_auc)